# Data Preparation

In [1]:
import pandas as pd
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft,fftfreq,fftshift
from numpy import genfromtxt
from PyIF import te_compute as te
import numpy as np
from pyinform import transfer_entropy as te
from os import listdir
from os.path import isfile, join

C:\Users\Ritwik\anaconda3\lib\site-packages\numba\core\decorators.py:232: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [2]:
import pandas as pd
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft,fftfreq,fftshift
from numpy import genfromtxt

from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_squared_error
from math import sqrt

import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm

# All the processes combined

In [3]:
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=2):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = signal.lfilter(b, a, data)
    return y

In [4]:
def loadmatrix(folder,subject):
    mat = scipy.io.loadmat(folder+'//'+subject)
    lis = list(mat.keys())
    keys = lis[2:]
    return mat,keys

In [5]:
def downsample(mat,vid):
    df1=pd.DataFrame(mat[keys[vid]])
    array = np.array(df1)
    print(np.shape(array))
#     ds = array[:,::5]
    ds = signal.decimate(array,5)
    print(np.shape(ds))
    dsdf = pd.DataFrame(ds)
    return dsdf

In [6]:
def filtsig(dsdf):
    ts = np.array(dsdf)
    lowcut = 0.5
    highcut = 60
    fs = 200
    filt = np.zeros(np.shape(ts))
    for i in range (0,len(ts)):
        filt[i] = butter_bandpass_filter(ts[i], lowcut, highcut, fs, order=2)
    fi = pd.DataFrame(filt)
    fiarr = np.array(fi)
    return fi,fiarr

In [7]:
def pickchannels():
    dfchan=pd.read_csv('C:/Users/Ritwik/Desktop/DSPmini/data/channels.csv', sep=',',header=None)
    channels = np.array(dfchan)
    indices = ['FP1','FP2','F7','F3','FZ','F4','F8','T7','C3','CZ','C4','T8','P7','P3','P4','P8','O1','OZ']
    chan18 = np.zeros(18)
    count = 0
    for ind in range (0,len(indices)):
        for cha in range(0,len(channels)):
            if(channels[cha] == indices[ind]):
                chan18[count] = int(cha)
                count+=1
                break
    df18 = fi.iloc[chan18]
    return df18,chan18


In [12]:
def VARInput(fiarr,chan18):
    framelength = 2*int(len(fiarr.T)/30)-1
    frameshift = int(len(fiarr.T)/30)

    Bigdf = np.empty((30,framelength))
    for i in chan18:
        start = 0
        arr = np.array(fi.iloc[int(i)])
        ch = np.empty((1,framelength))
        while((start)<len(arr)):
            if((framelength+start)>len(arr)):
                newarr = np.zeros(start+framelength-len(arr))
                ch1 = np.append(arr[start:len(arr)],newarr)
                ch=np.vstack((ch,ch1))
                break
            else:
                if(start == 0):
                    ch = np.vstack((ch,arr[start:(framelength+start)]))
                    ch = np.delete(ch,[0],axis=0)
                else:
                    ch = np.vstack((ch,arr[start:(framelength+start)]))

            start+=frameshift
        if(i==chan18[0]):
            Bigdf = np.dstack((Bigdf,ch))
            Bigdf = np.delete(Bigdf,[0],axis=2)
        else:
            Bigdf = np.dstack((Bigdf,ch))
    VAR_input = Bigdf
    print(np.shape(Bigdf))
    
    return VAR_input

In [46]:
def VARsig(VAR_input,epoch):
    model = VAR(VAR_input[epoch])
    results = model.fit(2)
    return results


In [11]:
def ARsig(AR_input,epoch):
    model = AR(AR_input[epoch])
    results = model.fit(2)
    return results

In [59]:
def GrangCaus(results):
    cov_matrix = results.acf(nlags=2)
    Covdf = pd.DataFrame(cov_matrix[1])
    cov = cov_matrix[0]
    GC = np.empty((18,18))
    for i in range(0,18):
        for j in range(0,18):
            GC[i][j] = np.log(np.abs(cov[i][j])/np.abs(cov[i][i]))
            if (GC[i][j] > 0):
                GC[i][j] = 1
            else:
                GC[i][j] = 0
    # print(GC)
    GCdf = pd.DataFrame(GC)
    return GC,GCdf

In [49]:
def TransferEntropy(GC):
    TE = np.empty(np.shape(GC))
    for i in range(0,len(GC)):
        for j in range(0,len(GC)):
            TE[i][j] = te(GC[i],GC[j],k=2)

    TEdf = pd.DataFrame(TE)
    return TE,TEdf

In [58]:
folder = "C://Users//Ritwik//Desktop//DSPMini//data//eeg_raw_data//1"
files = [f for f in listdir(folder) if isfile(join(folder, f))]
print(files)
# for index in len(files):
# subject = files[index]
count = 1
for x in range(0,1):
    mat,keys = loadmatrix(folder,subject)
    print(keys)
    for vid in range(1,2):                ###Keys variable has the matrix ids stored from index=1
        dsdf = downsample(mat,vid)
        fi,fiarr=filtsig(dsdf)
        print(np.shape(fiarr))
        df18,chan18 = pickchannels()
#         if(count<6):
#             pd.DataFrame.to_csv(df18,"C:\\Users\\Ritwik\\Desktop\\DSPmini\\exp\\downsampled_matrices\\"+"Sub"+str(subject[:2])+"\\"+keys[vid]+ ".csv")
#         elif(count>=6):
#             pd.DataFrame.to_csv(df18,"C:\\Users\\Ritwik\\Desktop\\DSPmini\\exp\\downsampled_matrices\\"+"Sub"+str(subject[:1])+"\\"+keys[vid]+ ".csv")
#             print(count)
        VAR_input = VARInput(fiarr,chan18)
        image_data.append(VAR_input)
#         print(np.shape(VAR_input))
        for epoch in range(0,1):
            results = VARsig(VAR_input,epoch)
#             print(results.summary())
            GC,GCdf = GrangCaus(VARsig(VAR_input,epoch))
#             if(count<6):
#                 pd.DataFrame.to_csv(GCdf,"C:\\Users\\Ritwik\\Desktop\\DSPmini\\exp\\causal_matrices\\"+"Sub"+str(subject[:2])+"\\"+keys[vid]+"_"+str(epoch)+".csv")
#             elif(count>=6):
#                 pd.DataFrame.to_csv(GCdf,"C:\\Users\\Ritwik\\Desktop\\DSPmini\\exp\\causal_matrices\\"+"Sub"+str(subject[:1])+"\\"+keys[vid]+"_"+str(epoch)+".csv")
            TE,TEdf = TransferEntropy(GC)
#             print(results.summary())
#             if(count<6):
#                 pd.DataFrame.to_csv(TEdf,"C:\\Users\\Ritwik\\Desktop\\DSPmini\\exp\\te_matrices\\"+"Sub"+str(subject[:2])+"\\"+keys[vid]+"_"+str(epoch)+".csv")
#             elif(count>=6):
#                 pd.DataFrame.to_csv(TEdf,"C:\\Users\\Ritwik\\Desktop\\DSPmini\\exp\\te_matrices\\"+"Sub"+str(subject[:1])+"\\"+keys[vid]+"_"+str(epoch)+".csv")
#             flat = TE.flatten()
#             np.savetxt("C:\\Users\\Ritwik\\Desktop\\DSPmini\\exp\\features\\"+"Sub"+str(sub)+"\\"+keys[vid]+"_"+str(epoch)+".csv",flat)
                   
    count+=1

##Store the features

['10_20151014.mat', '11_20150916.mat', '12_20150725.mat', '13_20151115.mat', '14_20151205.mat', '15_20150508.mat', '1_20160518.mat', '2_20150915.mat', '3_20150919.mat', '4_20151111.mat', '5_20160406.mat', '6_20150507.mat', '7_20150715.mat', '8_20151103.mat', '9_20151028.mat']
['__globals__', 'rx_eeg1', 'rx_eeg2', 'rx_eeg3', 'rx_eeg4', 'rx_eeg5', 'rx_eeg6', 'rx_eeg7', 'rx_eeg8', 'rx_eeg9', 'rx_eeg10', 'rx_eeg11', 'rx_eeg12', 'rx_eeg13', 'rx_eeg14', 'rx_eeg15', 'rx_eeg16', 'rx_eeg17', 'rx_eeg18', 'rx_eeg19', 'rx_eeg20', 'rx_eeg21', 'rx_eeg22', 'rx_eeg23', 'rx_eeg24']
(62, 33601)
(62, 6721)
(62, 6721)
(30, 447, 18)
(666, 666)


IndexError: invalid index to scalar variable.

In [16]:
print(len(image_data))

1


In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=2

test = 'ssr-chi2test'

def grangers_causality_matrix(data, variables, test = 'ssr_chi2test', verbose=False):

    dataset = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)

    for c in dataset.columns:
        for r in dataset.index:
            test_result = grangercausalitytests(data[[r,c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')

            min_p_value = np.min(p_values)
            dataset.loc[r,c] = min_p_value

    dataset.columns = [var + '_x' for var in variables]

    dataset.index = [var + '_y' for var in variables]

    return dataset

grangers_causality_matrix(dataset, variables = dataset.columns)  

In [ ]:
#epoch = 30
#vid = len(keys)

In [34]:
folder = "C://Users//Ritwik//Desktop//DSPMini//data//eeg_raw_data//1"
files = [f for f in listdir(folder) if isfile(join(folder, f))]
print(files[0])

count = 1
image_data = []
for x in range(0,len(files)):
    subject = files[x]
    mat,keys = loadmatrix(folder,subject)
    print(keys)
    for vid in range(1,len(keys)):
        dsdf = downsample(mat,vid)
        fi,fiarr=filtsig(dsdf)
        print(np.shape(fiarr))
        df18,chan18 = pickchannels()
#         if(count<6):
#             pd.DataFrame.to_csv(df18,"C:\\Users\\Ritwik\\Desktop\\DSPmini\\exp\\downsampled_matrices\\"+"Sub"+str(subject[:2])+"\\"+keys[vid]+ ".csv")
#         elif(count>=6):
#             pd.DataFrame.to_csv(df18,"C:\\Users\\Ritwik\\Desktop\\DSPmini\\exp\\downsampled_matrices\\"+"Sub"+str(subject[:1])+"\\"+keys[vid]+ ".csv")
#             print(count)
        VAR_input = VARInput(fiarr,chan18)
        image_data.append(VAR_input) 

10_20151014.mat
['__globals__', 'tyc_eeg1', 'tyc_eeg2', 'tyc_eeg3', 'tyc_eeg4', 'tyc_eeg5', 'tyc_eeg6', 'tyc_eeg7', 'tyc_eeg8', 'tyc_eeg9', 'tyc_eeg10', 'tyc_eeg11', 'tyc_eeg12', 'tyc_eeg13', 'tyc_eeg14', 'tyc_eeg15', 'tyc_eeg16', 'tyc_eeg17', 'tyc_eeg18', 'tyc_eeg19', 'tyc_eeg20', 'tyc_eeg21', 'tyc_eeg22', 'tyc_eeg23', 'tyc_eeg24']
(62, 33601)
(62, 6721)
(62, 6721)
(30, 447, 18)
(62, 19001)
(62, 3801)
(62, 3801)
(30, 251, 18)
(62, 39801)
(62, 7961)
(62, 7961)
(30, 529, 18)
(62, 26001)
(62, 5201)
(62, 5201)
(30, 345, 18)
(62, 17601)
(62, 3521)
(62, 3521)
(30, 233, 18)
(62, 32401)
(62, 6481)
(62, 6481)
(30, 431, 18)
(62, 30601)
(62, 6121)
(62, 6121)
(30, 407, 18)
(62, 41801)
(62, 8361)
(62, 8361)
(30, 555, 18)
(62, 29001)
(62, 5801)
(62, 5801)
(30, 385, 18)
(62, 33801)
(62, 6761)
(62, 6761)
(30, 449, 18)
(62, 10001)
(62, 2001)
(62, 2001)
(30, 131, 18)
(62, 22001)
(62, 4401)
(62, 4401)
(30, 291, 18)
(62, 43401)
(62, 8681)
(62, 8681)
(30, 577, 18)
(62, 33801)
(62, 6761)
(62, 6761)
(30, 44

(62, 8681)
(62, 8681)
(30, 577, 18)
(62, 33801)
(62, 6761)
(62, 6761)
(30, 449, 18)
(62, 51801)
(62, 10361)
(62, 10361)
(30, 689, 18)
(62, 28201)
(62, 5641)
(62, 5641)
(30, 375, 18)
(62, 13601)
(62, 2721)
(62, 2721)
(30, 179, 18)
(62, 35801)
(62, 7161)
(62, 7161)
(30, 475, 18)
(62, 28001)
(62, 5601)
(62, 5601)
(30, 371, 18)
(62, 9601)
(62, 1921)
(62, 1921)
(30, 127, 18)
(62, 22401)
(62, 4481)
(62, 4481)
(30, 297, 18)
(62, 22401)
(62, 4481)
(62, 4481)
(30, 297, 18)
(62, 35001)
(62, 7001)
(62, 7001)
(30, 465, 18)
(62, 27401)
(62, 5481)
(62, 5481)
(30, 363, 18)
['__globals__', 'cz_eeg1', 'cz_eeg2', 'cz_eeg3', 'cz_eeg4', 'cz_eeg5', 'cz_eeg6', 'cz_eeg7', 'cz_eeg8', 'cz_eeg9', 'cz_eeg10', 'cz_eeg11', 'cz_eeg12', 'cz_eeg13', 'cz_eeg14', 'cz_eeg15', 'cz_eeg16', 'cz_eeg17', 'cz_eeg18', 'cz_eeg19', 'cz_eeg20', 'cz_eeg21', 'cz_eeg22', 'cz_eeg23', 'cz_eeg24']
(62, 33601)
(62, 6721)
(62, 6721)
(30, 447, 18)
(62, 19001)
(62, 3801)
(62, 3801)
(30, 251, 18)
(62, 39801)
(62, 7961)
(62, 7961)
(30, 529, 

(30, 251, 18)
(62, 39801)
(62, 7961)
(62, 7961)
(30, 529, 18)
(62, 26001)
(62, 5201)
(62, 5201)
(30, 345, 18)
(62, 17601)
(62, 3521)
(62, 3521)
(30, 233, 18)
(62, 32401)
(62, 6481)
(62, 6481)
(30, 431, 18)
(62, 30601)
(62, 6121)
(62, 6121)
(30, 407, 18)
(62, 41801)
(62, 8361)
(62, 8361)
(30, 555, 18)
(62, 29001)
(62, 5801)
(62, 5801)
(30, 385, 18)
(62, 33801)
(62, 6761)
(62, 6761)
(30, 449, 18)
(62, 10001)
(62, 2001)
(62, 2001)
(30, 131, 18)
(62, 22001)
(62, 4401)
(62, 4401)
(30, 291, 18)
(62, 43401)
(62, 8681)
(62, 8681)
(30, 577, 18)
(62, 33801)
(62, 6761)
(62, 6761)
(30, 449, 18)
(62, 51801)
(62, 10361)
(62, 10361)
(30, 689, 18)
(62, 28201)
(62, 5641)
(62, 5641)
(30, 375, 18)
(62, 13601)
(62, 2721)
(62, 2721)
(30, 179, 18)
(62, 35801)
(62, 7161)
(62, 7161)
(30, 475, 18)
(62, 28001)
(62, 5601)
(62, 5601)
(30, 371, 18)
(62, 9601)
(62, 1921)
(62, 1921)
(30, 127, 18)
(62, 22401)
(62, 4481)
(62, 4481)
(30, 297, 18)
(62, 22401)
(62, 4481)
(62, 4481)
(30, 297, 18)
(62, 35001)
(62, 7001)
(6

In [43]:
print(np.shape(image_data[0]))

(30, 447, 18)


In [ ]:
folder = "C://Users//Ritwik//Desktop//DSPMini//data//eeg_raw_data//1"
onlyfiles = [f for f in listdir(folder) if isfile(join(folder, f))]
for subject in onlyfiles:
    mat = scipy.io.loadmat(folder+'//'+subject)
    lis = list(mat.keys())
    keys = lis[2:]
    vid = 1
    sub = 1
    for vid in range(1,len(keys)):
        df1=pd.DataFrame(mat[keys[vid]])
        array = np.array(df1)
        ds = array[:,::5]
        dsdf = pd.DataFrame(ds)

        ts = np.array(dsdf)
        lowcut = 0.5
        highcut = 60
        fs = 200
        filt = np.zeros(np.shape(ts))
        for i in range (0,len(ts)):
            filt[i] = butter_bandpass_filter(ts[i], lowcut, highcut, fs, order=2)
        fi = pd.DataFrame(filt)
        fiarr = np.array(fi)
        print(np.shape(fiarr))

        dfchan=pd.read_csv('C:/Users/Ritwik/Desktop/DSPmini/data/channels.csv', sep=',',header=None)
        channels = np.array(dfchan)
        indices = ['FP1','FP2','F7','F3','FZ','F4','F8','T7','C3','CZ','C4','T8','P7','P3','P4','P8','O1','OZ']
        chan18 = np.zeros(18)
        count = 0
        for ind in range (0,len(indices)):
            for cha in range(0,len(channels)):
                if(channels[cha] == indices[ind]):
                    chan18[count] = int(cha)
                    count+=1
                    break

        df18 = fi.iloc[chan18]
        pd.DataFrame.to_csv(df18,"C:\\Users\\Ritwik\\Desktop\\DSPmini\\exp\\downsampled_matrices\\"+"Sub"+str(subject[:2])+"\\"+keys[vid]+ ".csv")

        framelength = 2*int(len(fiarr.T)/30)-1
        frameshift = int(len(fiarr.T)/30)

        Bigdf = np.empty((30,framelength))
        for i in chan18:
            start = 0
            arr = np.array(fi.iloc[int(i)])
            ch = np.empty((1,framelength))
            while((start)<len(arr)):
                if((framelength+start)>len(arr)):
                    newarr = np.zeros(start+framelength-len(arr))
                    ch1 = np.append(arr[start:len(arr)],newarr)
                    ch=np.vstack((ch,ch1))
                    break
                else:
                    if(start == 0):
                        ch = np.vstack((ch,arr[start:(framelength+start)]))
                        ch = np.delete(ch,[0],axis=0)
                    else:
                        ch = np.vstack((ch,arr[start:(framelength+start)]))

                start+=frameshift
            if(i==chan18[0]):
                Bigdf = np.dstack((Bigdf,ch))
                Bigdf = np.delete(Bigdf,[0],axis=2)
            else:
                Bigdf = np.dstack((Bigdf,ch))
        VAR_input = Bigdf
        for epoch in range(0,30):
            model = VAR(VAR_input[epoch])
            results = model.fit(2)
            cov_matrix = results.acf(nlags = 2)
            Covdf = pd.DataFrame(cov_matrix[1])
            cov = cov_matrix[0]
            GC = np.empty((18,18))
            for i in range(0,18):
                for j in range(0,18):
                    GC[i][j] = np.log(np.abs(cov[i][j])/np.abs(cov[i][i]))
                    if (GC[i][j] > 0):
                        GC[i][j] = 1
                    else:
                        GC[i][j] = 0
            # print(GC)
            GCdf = pd.DataFrame(GC)
            pd.DataFrame.to_csv(GCdf,"C:\\Users\\Ritwik\\Desktop\\DSPmini\\exp\\causal_matrices\\"+"Sub"+str(subject)+"\\"+keys[vid]+"_"+str(epoch)+".csv")
            TE = np.empty(np.shape(GC))
            for i in range(0,len(GC)):
                for j in range(0,len(GC)):
                    TE[i][j] = te(GC[i],GC[j],k=2)

            TEdf = pd.DataFrame(TE)
            pd.DataFrame.to_csv(TEdf,"C:\\Users\\Ritwik\\Desktop\\DSPmini\\exp\\te_matrices\\"+"Sub"+str(subject)+"\\"+keys[vid]+"_"+str(epoch)+".csv")


In [ ]:
print(np.shape(VAR_input[0]))

In [ ]:
folder = "C://Users//Ritwik//Desktop//DSPMini//data//eeg_raw_data//1"
# onlyfiles = [f for f in listdir(folder) if isfile(join(folder, f))]
# print(onlyfiles)
mat = scipy.io.loadmat(folder+'//'+'1_20160518.mat')
mat = scipy.io.loadmat(onlyfiles)
lis = list(mat.keys())
keys = lis[2:]
df1=pd.DataFrame(mat[keys[1]])
array = np.array(df1)
ds = array[:,::5]
dsdf = pd.DataFrame(ds)


In [ ]:
sub = onlyfiles[9]
print(sub)

In [ ]:
for epoch in range(0,30):
    model = VAR(VAR_input[epoch])
    results = model.fit(2)
    cov_matrix = results.acf(nlags = 2)
    Covdf = pd.DataFrame(cov_matrix[0])
    cov = cov_matrix[0]
    GC = np.empty((18,18))
    for i in range(0,18):
        for j in range(0,18):
            GC[i][j] = np.log(np.abs(cov[i][j])/np.abs(cov[i][i]))
            if (GC[i][j] > 0):
                GC[i][j] = 1
            else:
                GC[i][j] = 0
    # print(GC)
    GCdf = pd.DataFrame(GC)
    pd.DataFrame.to_csv(GCdf,"C:\\Users\\Ritwik\\Desktop\\DSPmini\\exp\\causal_matrices\\"+keys[2]+str(epoch)+".csv")
    TE = np.empty(np.shape(GC))
    for i in range(0,len(GC)):
        for j in range(0,len(GC)):
            TE[i][j] = te(GC[i],GC[j],k=2)

    TEdf = pd.DataFrame(TE)
    pd.DataFrame.to_csv(TEdf,"C:\\Users\\Ritwik\\Desktop\\DSPmini\\exp\\te_matrices\\"+keys[2]+"_"+str(epoch)+".csv")

# VAR

In [ ]:
import pandas as pd
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.fft import fft,fftfreq,fftshift
from numpy import genfromtxt

from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [ ]:
epoch = 0
model = VAR(VAR_input[0])
print(np.shape(VAR_input[0]))

In [ ]:
results = model.fit(2)
results.summary()

In [ ]:
cov_matrix = results.acf(nlags = 2)

In [ ]:
Covdf = pd.DataFrame(cov_matrix[0])
cov = cov_matrix[0]
Covdf.head(18)

# Granger Causality

In [ ]:
GC = np.empty((18,18))
for i in range(0,18):
    for j in range(0,18):
        GC[i][j] = np.log(np.abs(cov[i][j])/np.abs(cov[i][i]))
        if (GC[i][j] > 0):
            GC[i][j] = 1
        else:
            GC[i][j] = 0
print(GC)
GCdf = pd.DataFrame(GC)

GCdf.head(20)

In [ ]:
pd.DataFrame.to_csv(GCdf,"C:\\Users\\Ritwik\\Desktop\\DSPmini\\exp\\causal_matrices\\cz_eeg1_"+str(epoch)+".csv")

# Transfer Entropy

In [ ]:
TE = np.empty(np.shape(GC))
for i in range(0,len(GC)):
    for j in range(0,len(GC)):
        TE[i][j] = te(GC[i],GC[j],k=2)

TEdf = pd.DataFrame(TE)
TEdf.head(30)

In [ ]:
pd.DataFrame.to_csv(TEdf,"C:\\Users\\Ritwik\\Desktop\\DSPmini\\exp\\te_matrices\\cz_eeg1_0.csv")

# Classifier

In [ ]:
folder = "C://Users//Ritwik//Desktop//DSPMini//data//eeg_raw_data//1"
# onlyfiles = [f for f in listdir(folder) if isfile(join(folder, f))]
# print(onlyfiles)
mat = scipy.io.loadmat(folder+'//'+'1_20160518.mat')
# mat = scipy.io.loadmat(onlyfiles)
lis = list(mat.keys())
keys = lis[2:]

In [ ]:
sub = 1
for vid in range(1,len(keys)):
    for epoch in range(0,30):
#         print(keys[vid])
        feat = pd.read_csv("C:\\Users\\Ritwik\\Desktop\\DSPmini\\exp\\te_matrices\\"+"Sub"+str(sub)+"\\"+keys[vid]+"_"+str(epoch)+".csv",index_col = 0)
        flat = np.array(feat).flatten()
        np.savetxt("C:\\Users\\Ritwik\\Desktop\\DSPmini\\exp\\features\\"+"Sub"+str(sub)+"\\"+keys[vid]+"_"+str(epoch)+".csv",flat)
    

In [ ]:
import numpy as np
featdim =324
def load_input(file_path):
    with open(file_path) as f:  
        line = f.readline()
        cnt = 1
        tr = np.empty(shape=[0,featdim])
        emotion_trgt = []
        while line:
            [path,rem] = line.split()
#             print(path)
            emotion = float(rem)
            with open(path, 'rb') as fid:
                data_array = np.fromfile(fid, sep= '\n')
#             print("data array",data_array)
            da = np.reshape(data_array,(1,-1))
        #    break
            emotion_trgt.append(emotion)
            tr = np.append(tr,da,axis=0)
            line = f.readline()
            cnt+=1
        print("train shape :",np.shape(tr), "targets shape :", np.shape(emotion_trgt))
    return [tr,emotion_trgt]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
[X,Y]=load_input("C:\\Users\\Ritwik\\Desktop\\DSPmini\\exp\\features\\Sub1.txt")

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=20)

In [ ]:
#Import svm model
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc,accuracy_score
import pylab as pl
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline

clf = make_pipeline(StandardScaler(), svm.SVC(gamma='auto'))
clf = svm.SVC(gamma = 'scale',kernel='rbf')
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svc', svm.SVC(gamma='auto'))])
# clf = svm.LinearSVC(random_state = 0,tol=1e-5,class_weight = 'balanced')
C = [0.01,0.1,1,10,100]
for i in C:
    clf.C = i
    clf.fit(X_train,Y_train)
#     acc=clf.score(X_train,Y_train)
    y_pred = clf.predict(X_test)
    Conf = confusion_matrix(Y_test, y_pred)
#     accu = clf.score(y_pred,Y_test)
#     probas_ = clf.fit(X_train,Y_train).predict_proba(X_test)
#     fpr, tpr, thresholds = roc_curve(Yt, probas_[:, 1])
#     roc_auc = auc(fpr, tpr1)
#     print ("Area under the ROC curve : %f" % roc_auc)
#     pl.clf()
#     pl.plot(fpr1, tpr1, label='ROC curve (area = %0.2f)' % roc_auc)
#     pl.plot([0, 1], [0, 1], 'k--')
#     pl.xlim([0.0, 1.0])
#     pl.ylim([0.0, 1.0])
#     pl.xlabel('False Positive Rate')
#     pl.ylabel('True Positive Rate')
#     pl.title('Receiverrating characteristic example')
#     pl.legend(loc="lower right")
#     print(accu)
#     print(Conf)
    print("Accuracy:",accuracy_score(Y_test,y_pred))